# **Create your own layer** - *Inspiré de la formation de Thibault Neveu*
We are going to create a custom layer. The goal of this layer is to create a multi layer perceptron. By doing so, we can also learn how to use some low level operation with tensorflow 2.0.

# 0. Imports

In [1]:
import tensorflow as tf
import numpy as np

# 1. Be sure to use Tensor Flow 2.0

In [2]:
assert hasattr(tf, "function") # Be sure to use tensorflow 2.0

# 2. Multi Layer Perceptron

This layer has no real purpose and should never be used in production. This is just an example to show how to create a custom layer.

In [3]:
class MlpLayer(tf.keras.layers.Layer):

    def __init__(self, units, activations, **kwargs):
        super(MlpLayer, self).__init__(**kwargs)
        # Set the property to the layer
        self.units = units
        self.activations_list = activations
        self.weights_list = []

    # The build method will be called once
    # we know the shape of the previous Layer: input_shape
    def build(self, input_shape):
        # Create trainable weights variables for this layer.
        # We create matrix of weights for each layer
        # Each weight have this shape: (previous_layer_size, layer_size)
        i = 0
        for units in self.units:
            weights = self.add_weight(
                        name='weights-%s' % i,
                        shape=(input_shape[1], units),
                        initializer='uniform',
                        trainable=True
            )
            i += 1
            self.weights_list.append(weights)
            input_shape = (None, units)
        super(MlpLayer, self).build(input_shape)

    def call(self, x):
        output = x

        # We go through each weight to compute the dot product between the previous
        # activation and the weight of the layer.
        # At the first pass, the previous activation is just the variable "x": The input vector
        for weights, activation in zip(self.weights_list, self.activations_list):
            # We can still used low level operations as tf.matmul, tf.nn.relu...
            output = tf.matmul(output, weights)

            if activation == "relu":
                output = tf.nn.relu(output)
            elif activation == "sigmoid":
                output = tf.nn.sigmoid(output)
            elif activation == "softmax":
                output = tf.nn.softmax(output)

        return output

    # By adding the get_config method you can then save your model with the custom layer
    # and retrieve the model with the same parameters
    def get_config(self):
        config = {
            'units': self.units,
            "activations": self.activations_list
        }
        # Retrieve the config from the parent layer
        base_config = super(MlpLayer, self).get_config()
        # Return the final config
        return dict(list(base_config.items()) + list(config.items()))

# Flatten
model = tf.keras.models.Sequential()

# Add the layers
model.add(MlpLayer([4 , 2], ["relu", "softmax"]))
model.predict(np.zeros((5, 10)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 882ms/step


array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32)

Pour mieux comprendre comment l'exemple fourni fonctionne, examinons pas à pas ce qui se passe lorsque vous utilisez la couche personnalisée MlpLayer dans un modèle séquentiel et faites une prédiction sur des données d'entrée.

----------------------------------------

Mise en place de l'exemple

Création du modèle séquentiel

model = tf.keras.models.Sequential()
Un modèle Keras séquentiel est créé, ce qui signifie que les couches seront empilées linéairement les unes après les autres.
Ajout de la couche personnalisée MlpLayer

model.add(MlpLayer([4, 2], ["relu", "softmax"]))
La couche personnalisée MlpLayer est ajoutée au modèle avec deux sous-couches :
Première sous-couche : 4 neurones avec la fonction d'activation ReLU.
Deuxième sous-couche : 2 neurones avec la fonction d'activation Softmax.
Prédiction sur un échantillon d'entrée

model.predict(np.zeros((5, 10)))
Le modèle effectue une prédiction sur un lot d'entrée de forme (5, 10), ce qui signifie 5 exemples, chacun avec 10 caractéristiques.
Déroulement des calculs
Initialisation et construction des poids
Initialisation (__init__) :

units = [4, 2] et activations = ["relu", "softmax"] sont définis.
weights_list est initialisé en tant que liste vide pour stocker les matrices de poids de chaque sous-couche.
Construction (build) :

----------------------------------------


Pour la première sous-couche (4 neurones) :
La taille d'entrée est (10,) (10 caractéristiques).
Une matrice de poids est créée de forme (10, 4), initialisée uniformément.
Ces poids sont ajoutés à weights_list.
La taille de sortie est mise à jour à (4,).
Pour la deuxième sous-couche (2 neurones) :
La taille d'entrée devient (4,) (4 caractéristiques, sortie de la première sous-couche).
Une matrice de poids est créée de forme (4, 2), initialisée uniformément.
Ces poids sont ajoutés à weights_list.
Passe avant (call)
Entrée Initiale :

L'entrée x a la forme (5, 10), soit 5 exemples avec 10 caractéristiques chacun, initialisées à zéro.
Première Sous-couche :

Produit Matriciel : output = tf.matmul(x, weights_1) où weights_1 a la forme (10, 4).
Chaque exemple est transformé en un vecteur de 4 valeurs à l'aide des poids.
Activation ReLU : output = tf.nn.relu(output)
Applique ReLU, transformant les valeurs négatives en zéro, pour chaque élément du vecteur de sortie.
Deuxième Sous-couche :

Produit Matriciel : output = tf.matmul(output, weights_2) où weights_2 a la forme (4, 2).
Le vecteur de 4 valeurs est transformé en un vecteur de 2 valeurs.
Activation Softmax : output = tf.nn.softmax(output)
Convertit le vecteur de sortie en une distribution de probabilité (somme à 1).
Résultat de la Prédiction
Le modèle renvoie une matrice de sortie de forme (5, 2), où chaque ligne représente un exemple et chaque colonne représente la probabilité d'appartenance à l'une des deux classes.


----------------------------------------

Cet exemple montre comment la couche MlpLayer prend une entrée de 10 caractéristiques, la transforme à travers deux sous-couches définies par les paramètres [4, 2] et les fonctions d'activation ["relu", "softmax"], et produit une sortie qui est une distribution de probabilité sur deux classes pour chaque exemple. Chaque étape, de l'initialisation à la passe avant, est soigneusement orchestrée pour respecter les dimensions et appliquer les transformations souhaitées.

In [4]:
model.save("custom_layer_in_model.h5")

model.save("custom_layer_in_model.keras")

In [5]:
custom_objects={'MlpLayer': MlpLayer}
loaded_model = tf.keras.models.load_model("custom_layer_in_model.h5", custom_objects=custom_objects)

In [6]:
# Chargement du modèle depuis le fichier .keras
loaded_model = tf.keras.models.load_model("custom_layer_in_model.keras", custom_objects={'MlpLayer': MlpLayer})

# Compiler le modèle après le chargement si vous prévoyez de continuer l'entraînement
loaded_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)